In [1]:
import math
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../data/Chembl_bioactivity_SIRT1.tsv", delimiter="\t")

In [3]:
df.columns

Index(['CMPD_CHEMBLID', 'MOLREGNO', 'PARENT_CMPD_CHEMBLID', 'PARENT_MOLREGNO',
       'MOL_PREF_NAME', 'COMPOUND_KEY', 'MOLWEIGHT', 'ALOGP', 'PSA',
       'NUM_RO5_VIOLATIONS', 'CANONICAL_SMILES', 'ACTIVITY_ID',
       'STANDARD_TYPE', 'RELATION', 'STANDARD_VALUE', 'STANDARD_UNITS',
       'PCHEMBL_VALUE', 'PUBLISHED_TYPE', 'PUBLISHED_RELATION',
       'PUBLISHED_VALUE', 'PUBLISHED_UNITS', 'ACTIVITY_COMMENT',
       'DATA_VALIDITY_COMMENT', 'POTENTIAL_DUPLICATE', 'BAO_ENDPOINT',
       'UO_UNITS', 'QUDT_UNITS', 'ASSAY_ID', 'ASSAY_CHEMBLID', 'ASSAY_TYPE',
       'DESCRIPTION', 'ASSAY_SRC_ID', 'ASSAY_SRC_DESCRIPTION',
       'ASSAY_ORGANISM', 'ASSAY_STRAIN', 'ASSAY_TAX_ID', 'CURATED_BY',
       'BAO_FORMAT', 'TID', 'TARGET_CHEMBLID', 'TARGET_TYPE',
       'PROTEIN_ACCESSION', 'PREF_NAME', 'ORGANISM', 'CONFIDENCE_SCORE',
       'TARGET_MAPPING', 'APD_NAME', 'APD_CONFIDENCE', 'DOC_ID',
       'DOC_CHEMBLID', 'PUBMED_ID', 'JOURNAL', 'YEAR', 'VOLUME', 'ISSUE',
       'FIRST_PAGE', 'CELL_ID',

In [4]:
df = df[["CMPD_CHEMBLID", "DOC_CHEMBLID", "CANONICAL_SMILES", "STANDARD_TYPE", "RELATION", "STANDARD_VALUE", "STANDARD_UNITS"]]

In [5]:
df_ic50 = df[df.STANDARD_TYPE == "IC50"]

In [6]:
df_ic50_uniq = df_ic50.drop_duplicates(subset=["CMPD_CHEMBLID", "DOC_CHEMBLID"])

In [7]:
df_ic50_uniq.sort_values(by=["DOC_CHEMBLID", "CMPD_CHEMBLID"])

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS
1884,CHEMBL218785,CHEMBL1137294,CCCCCCCn1cc(C2=C(C(=O)N(C)C2=O)c3cn(CCCCCCC)c4...,IC50,NaN,NaN,NaN
379,CHEMBL291725,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2c[nH]c3ccccc23)c4c[nH]c5cc...,IC50,>,50000.0,nM
1319,CHEMBL296586,CHEMBL1137294,Brc1ccc2[nH]c3c(CC(=O)Nc4ccccc34)c2c1,IC50,NaN,NaN,NaN
614,CHEMBL298346,CHEMBL1137294,O=C1N(Cc2ccccc2)C(=O)C(=C1c3c[nH]c4ccccc34)c5c...,IC50,NaN,NaN,NaN
639,CHEMBL302449,CHEMBL1137294,Cn1c2ccccc2c3c4C(=O)NCc4c5c6ccccc6n(CCC#N)c5c13,IC50,NaN,NaN,NaN
643,CHEMBL380797,CHEMBL1137294,CC(NC(=O)c1ccccc1\N=C\c2c(O)ccc3ccccc23)c4ccccc4,IC50,NaN,NaN,NaN
467,CHEMBL384899,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(Cc3ccccc3)c4ccccc24)c5c...,IC50,NaN,NaN,NaN
425,CHEMBL387119,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(CCCCCCCO)c3ccccc23)c4cn...,IC50,NaN,NaN,NaN
1735,CHEMBL388978,CHEMBL1137294,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n3c4ccccc4...,IC50,NaN,NaN,NaN
103,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,IC50,=,50000.0,nM


In [8]:
def ic50_um_to_pic50(ic50_um):
    return - (np.log10(ic50_um) - 6)

def ic50_to_relevance_score(relation, standard_value, standard_unit, ic50_um_threshold=100):
    """
    relevance_score: large -> good compound
    relation: 
    * "=": pIC50 (ic50_um < ic50_um_threshold) else 0
    * ">": 0 (ic50_um >= ic50_um_threshold) else None
    * "<": pIC50 (ic50_um < ic50_um_threshold) else None
    * "NaN": 0 (the compound may not be inhibitor)
    """
    if standard_unit == "nM":
        ic50_um = standard_value * 1e-3
    elif isinstance(standard_unit, float) and math.isnan(standard_unit):
        return 0
    else:
        return None
    
    if relation == "=":
        if ic50_um < ic50_um_threshold:
            return ic50_um_to_pic50(ic50_um)
        else:
            return 0
    elif relation == ">":
        if ic50_um >= ic50_um_threshold:
            return 0
        else:
            return None
    elif relation == "<":
        if ic50_um < ic50_um_threshold:
            return ic50_um_to_pic50(ic50_um)
        else:
            return None
    else:
        raise ValueError("{} {} {}".format(relation, standard_value, standard_unit))

In [9]:
df_ic50_uniq["relevance"] = df_ic50_uniq.apply(lambda row: ic50_to_relevance_score(row.RELATION, row.STANDARD_VALUE, row.STANDARD_UNITS), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [10]:
df_ic50_uniq.sort_values(by="relevance", ascending=False)

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance
1357,CHEMBL2332039,CHEMBL2331260,CCNC(=O)c1ccc(s1)C(=O)NCCC2CCN(CC2)c3ncnc4cc(s...,IC50,=,3.6,nM,8.443697
749,CHEMBL2338810,CHEMBL2331260,CS(=O)(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,4.3,nM,8.366532
1078,CHEMBL2332037,CHEMBL2331260,CCNC(=O)c1cccc(c1)C(=O)NCCC2CCN(CC2)c3ncnc4cc(...,IC50,=,5.3,nM,8.275724
544,CHEMBL2332043,CHEMBL2331260,NC(=O)c1cc2ncnc(N3CCC(CCNC(=O)c4cccs4)CC3)c2s1,IC50,=,5.8,nM,8.236572
1114,CHEMBL2332041,CHEMBL2331260,CC(C)(C)OC(=O)c1ccc(s1)C(=O)NCCC2CCN(CC2)c3ncn...,IC50,=,6.7,nM,8.173925
1299,CHEMBL2332042,CHEMBL2331260,NC(=O)c1cc2ncnc(N3CCC(CCNC(=O)c4ccc(s4)C(=O)O)...,IC50,=,14.0,nM,7.853872
2070,CHEMBL2332055,CHEMBL2331260,CC(C)(C)C(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,15.0,nM,7.823909
884,CHEMBL2332044,CHEMBL2331260,CC(C)(C)OC(=O)NCCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,17.0,nM,7.769551
146,CHEMBL2332040,CHEMBL2331260,CCNC(=O)c1ccc(s1)C(=O)NCCN2CCN(CC2)c3ncnc4cc(s...,IC50,=,31.0,nM,7.508638
1320,CHEMBL420311,CHEMBL1795336,NC(=O)C1CCCc2c1[nH]c3ccc(Cl)cc23,IC50,=,38.0,nM,7.420216


In [11]:
df_ic50_uniq.relevance.describe(include="all")

count    669.000000
mean       3.286927
std        2.543104
min        0.000000
25%        0.000000
50%        4.330683
75%        5.050610
max        8.443697
Name: relevance, dtype: float64

In [12]:
df_inhibition = df[df.STANDARD_TYPE == "Inhibition"]

In [13]:
df_inhibition_uniq = df_inhibition.drop_duplicates(subset=["CMPD_CHEMBLID", "DOC_CHEMBLID"])

In [14]:
df_inhibition_uniq.sort_values(by=["DOC_CHEMBLID", "CMPD_CHEMBLID"])

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS
1282,CHEMBL1140,CHEMBL1137294,NC(=O)c1cccnc1,Inhibition,=,10.6,%
1066,CHEMBL218328,CHEMBL1137294,CN1CCCC1CCn2cc(C3=C(C(=O)NC3=O)c4c[nH]c5ccccc4...,Inhibition,=,58.2,%
2028,CHEMBL218910,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4cccc(F)c...,Inhibition,=,29.9,%
1435,CHEMBL220456,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4cccc(F)c...,Inhibition,=,71.9,%
2092,CHEMBL263467,CHEMBL1137294,Fc1cccc2c1c3cn2CCO[C@@H](CCn4cc(C5=C3C(=O)NC5=...,Inhibition,=,55.7,%
943,CHEMBL266487,CHEMBL1137294,O=C1NC(=O)C(=C1c2c[nH]c3ccccc23)c4c[nH]c5ccccc45,Inhibition,=,52.7,%
1178,CHEMBL413219,CHEMBL1137294,OCCC(OCCn1cc(C2=C(C(=O)NC2=O)c3c[nH]c4ccccc34)...,Inhibition,=,77.5,%
1967,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,Inhibition,=,49.4,%
1729,CHEMBL443108,CHEMBL1138523,Fc1ccccc1C(=O)C2=Cc3c(OC2=O)ccc4ccccc34,Inhibition,=,3.3,%
504,CHEMBL447256,CHEMBL1138523,Clc1cccc(c1)C(=O)C2=Cc3c(OC2=O)ccc4ccccc34,Inhibition,=,3.7,%


In [15]:
def inhibition_to_relevance_score(relation, standard_value, standard_unit):
    """
    relevance_score: large -> good compound
    relation:
    * "=": max(standard_value, 0)
    * ">": max(standard_value, 0)
    * "<": None
    * "NaN": 0 (the compound may not be inhibitor)
    """
    if standard_unit != "%":
        return None
    elif isinstance(standard_unit, float) and math.isnan(standard_unit):
        return 0
    
    if relation == "=":
        return max(standard_value, 0)
    elif relation == ">":
        return max(standard_value, 0)
    elif relation == "<":
        return None
    else:
        raise ValueError("{} {} {}".format(relation, standard_value, standard_unit))

In [16]:
df_inhibition_uniq["relevance"] = df_inhibition_uniq.apply(lambda row: inhibition_to_relevance_score(row.RELATION, row.STANDARD_VALUE, row.STANDARD_UNITS), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
df_inhibition_uniq.sort_values(by="relevance", ascending=False)

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance
688,CHEMBL2420981,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCc1ccccc1)C(=O)NCc2...,Inhibition,=,100.0,%,100.0
1463,CHEMBL1254033,CHEMBL1250463,C[C@H](NC(=O)[C@H](CCCCNC(=[Se])C)NC(=O)[C@H](...,Inhibition,=,100.0,%,100.0
2010,CHEMBL1254613,CHEMBL1250463,CC(C)CC(=S)NCCCC[C@H](NC(=O)[C@H](C)NC(=O)C)C(...,Inhibition,=,100.0,%,100.0
596,CHEMBL1254032,CHEMBL1250463,C[C@H](NC(=O)[C@H](CCCCNC(=S)C)NC(=O)[C@H](C)N...,Inhibition,=,100.0,%,100.0
135,CHEMBL2420982,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCCc1ccccc1)C(=O)NCc...,Inhibition,=,99.0,%,99.0
1671,CHEMBL420311,CHEMBL1144627,NC(=O)C1CCCc2c1[nH]c3ccc(Cl)cc23,Inhibition,=,98.9,%,98.9
333,CHEMBL1254035,CHEMBL1250463,CC(=[Se])NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)Nc2c...,Inhibition,=,98.0,%,98.0
1588,CHEMBL2417693,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCc1ccccc1)C(=O)NCC(=O...,Inhibition,=,98.0,%,98.0
1745,CHEMBL2417694,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)CCCCCc1ccccc1)C(=O)NCC(...,Inhibition,=,98.0,%,98.0
35,CHEMBL1835312,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)NCC(=O)...,Inhibition,=,97.0,%,97.0


In [18]:
df_inhibition_uniq.relevance.describe(include="all")

count    473.000000
mean      36.228985
std       28.444874
min        0.000000
25%       12.000000
50%       29.400000
75%       54.800000
max      100.000000
Name: relevance, dtype: float64

In [19]:
# qid
"""
qid: (SIRT_N, {IC50|INHIBITION}, DOC_ID)
"""
doc_set = set(list(df_ic50_uniq.DOC_CHEMBLID) + list(df_inhibition_uniq.DOC_CHEMBLID))
doc_ind_map = {}
for ind, doc in enumerate(doc_set):
    doc_ind_map[doc] = ind + 1

In [20]:
df_ic50_uniq["qid"] = df_ic50_uniq.apply(lambda row: "1" + "1" + str(doc_ind_map[row.DOC_CHEMBLID]), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [21]:
df_ic50_uniq.sort_values(by="DOC_CHEMBLID")

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
103,CHEMBL430574,CHEMBL1137294,O=C1Cc2c([nH]c3ccccc23)c4ccccc4N1,IC50,=,50000.0,nM,4.301030,1145
614,CHEMBL298346,CHEMBL1137294,O=C1N(Cc2ccccc2)C(=O)C(=C1c3c[nH]c4ccccc34)c5c...,IC50,NaN,NaN,NaN,0.000000,1145
1884,CHEMBL218785,CHEMBL1137294,CCCCCCCn1cc(C2=C(C(=O)N(C)C2=O)c3cn(CCCCCCC)c4...,IC50,NaN,NaN,NaN,0.000000,1145
425,CHEMBL387119,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(CCCCCCCO)c3ccccc23)c4cn...,IC50,NaN,NaN,NaN,0.000000,1145
639,CHEMBL302449,CHEMBL1137294,Cn1c2ccccc2c3c4C(=O)NCc4c5c6ccccc6n(CCC#N)c5c13,IC50,NaN,NaN,NaN,0.000000,1145
643,CHEMBL380797,CHEMBL1137294,CC(NC(=O)c1ccccc1\N=C\c2c(O)ccc3ccccc23)c4ccccc4,IC50,NaN,NaN,NaN,0.000000,1145
1735,CHEMBL388978,CHEMBL1137294,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n3c4ccccc4...,IC50,NaN,NaN,NaN,0.000000,1145
1271,CHEMBL98275,CHEMBL1137294,ONC1=Nc2ccccc2c3[nH]c4ccc(Br)cc4c3C1,IC50,NaN,NaN,NaN,0.000000,1145
467,CHEMBL384899,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2cn(Cc3ccccc3)c4ccccc24)c5c...,IC50,NaN,NaN,NaN,0.000000,1145
379,CHEMBL291725,CHEMBL1137294,CN1C(=O)C(=C(C1=O)c2c[nH]c3ccccc23)c4c[nH]c5cc...,IC50,>,50000.0,nM,NaN,1145


In [22]:
df_inhibition_uniq["qid"] = df_inhibition_uniq.apply(lambda row: "1" + "2" + str(doc_ind_map[row.DOC_CHEMBLID]), axis=1)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [23]:
df_inhibition_uniq.sort_values(by="qid")

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
1683,CHEMBL3350584,CHEMBL1773060,NC(=O)c1ccc[n+](c1)[C@@H]2O[C@H](COP(=O)([O-])...,Inhibition,=,31.0,%,31.0,1210
1365,CHEMBL3350589,CHEMBL1773060,NC(=O)c1ccc[n+](c1)[C@@H]2O[C@H](COP(=O)([O-])...,Inhibition,=,16.0,%,16.0,1210
489,CHEMBL1775009,CHEMBL1773060,Nc1ncnc2c1nc(Br)n2[C@@H]3O[C@H](CO)[C@@H](O)[C...,Inhibition,<,10.0,%,NaN,1210
1449,CHEMBL1775010,CHEMBL1773060,Nc1ncnc2c1nc(c3ccccc3)n2[C@@H]4O[C@H](CO)[C@@H...,Inhibition,<,10.0,%,NaN,1210
366,CHEMBL3350587,CHEMBL1773060,COc1ccc(cc1)c2nc3c(N)ncnc3n2[C@@H]4O[C@H](COP(...,Inhibition,=,18.0,%,18.0,1210
1885,CHEMBL1230617,CHEMBL1773060,Nc1ncnc2c1nc(Br)n2[C@@H]3O[C@H](COP(=O)(O)O)[C...,Inhibition,<,10.0,%,NaN,1210
888,CHEMBL3350586,CHEMBL1773060,NC(=O)c1ccc[n+](c1)[C@@H]2O[C@H](COP(=O)([O-])...,Inhibition,=,30.0,%,30.0,1210
1902,CHEMBL3350588,CHEMBL1773060,Cc1ccc(cc1)c2nc3c(N)ncnc3n2[C@@H]4O[C@H](COP(=...,Inhibition,=,20.0,%,20.0,1210
472,CHEMBL3350585,CHEMBL1773060,NC(=O)c1ccc[n+](c1)[C@@H]2O[C@H](COP(=O)([O-])...,Inhibition,=,17.0,%,17.0,1210
509,CHEMBL1801799,CHEMBL1811898,Cc1ccc(NC(=O)c2cc(Cl)cc(Cl)c2O)cc1,Inhibition,NaN,NaN,NaN,NaN,1211


In [27]:
df_inhibition_uniq.dropna(subset=["relevance"], inplace=True)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [28]:
df_inhibition_uniq

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
0,CHEMBL511214,CHEMBL1155918,Fc1ccc(NC(=S)NNC(=O)c2onc(n2)c3cccc4ccccc34)cc1,Inhibition,=,10.00,%,10.00,1216
3,CHEMBL1835299,CHEMBL1833937,C[C@H](NC(=O)[C@@H](N)CCCCNC(=S)C)C(=O)O,Inhibition,=,46.90,%,46.90,1273
5,CHEMBL3233748,CHEMBL3232806,CC(C)c1ccc(cc1)C2=C(Cc3c(O)ccc4nc(Cl)ccc34)C(=...,Inhibition,=,85.00,%,85.00,1272
6,CHEMBL3290219,CHEMBL3286262,CCOC(=O)c1ccc2c(c1)nc(c3ccc(O)cc3)n2c4ccccc4,Inhibition,=,11.01,%,11.01,122
8,CHEMBL3237038,CHEMBL3232865,CN(C(=O)c1cccc(c1)S(=O)(=O)N(C)c2ccc(Br)cc2)c3...,Inhibition,=,0.00,%,0.00,124
11,CHEMBL1797747,CHEMBL1795290,Oc1ccc2ccccc2c1CC3=C(Cc4ccccc4)NC(=S)NC3=O,Inhibition,=,82.40,%,82.40,1213
30,CHEMBL3236707,CHEMBL3232865,CN(c1ccc(Cl)cc1)S(=O)(=O)c2cccc(c2)C(=O)Nc3ccc...,Inhibition,=,2.00,%,2.00,124
33,CHEMBL3580806,CHEMBL3580657,[O-][S+](Cc1ccc(Cl)cc1)c2cccc(c2)C(=O)Nc3ccc(c...,Inhibition,=,12.00,%,12.00,1218
35,CHEMBL1835312,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)NCC(=O)...,Inhibition,=,97.00,%,97.00,1219
37,CHEMBL2420991,CHEMBL2417552,CC(=S)NCCCC[C@H](NC(=O)[C@@H]1C[C@@H](O)CN1C(=...,Inhibition,=,90.00,%,90.00,1219


In [29]:
df_ic50_uniq.dropna(subset=["relevance"], inplace=True)

/Users/shogo/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
df_ic50_uniq

,CMPD_CHEMBLID,DOC_CHEMBLID,CANONICAL_SMILES,STANDARD_TYPE,RELATION,STANDARD_VALUE,STANDARD_UNITS,relevance,qid
1,CHEMBL3311063,CHEMBL3351989,CCOc1ccccc1c2nn3c(nnc3s2)c4occc4,IC50,=,23600.0,nM,4.627088,1144
4,CHEMBL3233748,CHEMBL3232806,CC(C)c1ccc(cc1)C2=C(Cc3c(O)ccc4nc(Cl)ccc34)C(=...,IC50,=,21000.0,nM,4.677781,1172
9,CHEMBL3408982,CHEMBL3407343,CCCCCCCCn1c2CCN(CCO)Cc2c3cc(ccc13)c4cnc(N)nc4,IC50,NaN,NaN,NaN,0.000000,1120
10,CHEMBL1140,CHEMBL3390834,NC(=O)c1cccnc1,IC50,=,25000.0,nM,4.602060,119
12,CHEMBL2332054,CHEMBL2331260,CC(C)(C)C(=O)NCC1CCN(CC1)c2ncnc3cc(sc23)C(=O)N,IC50,=,4300.0,nM,5.366532,1156
14,CHEMBL1140,CHEMBL3407343,NC(=O)c1cccnc1,IC50,=,149000.0,nM,0.000000,1120
15,CHEMBL3408831,CHEMBL3407343,CCCCCCCCn1c2CCNCc2c3cc(ccc13)c4cccc(C)c4,IC50,=,20000.0,nM,4.698970,1120
16,CHEMBL584353,CHEMBL1155190,CC(=S)NCCCC[C@H](NC(=O)OCc1ccccc1)C(=O)Nc2ccccc2,IC50,=,2700.0,nM,5.568636,1160
17,CHEMBL1254694,CHEMBL1250463,COC(=O)[C@H](C)NC(=O)[C@H](CCCCNC(=O)c1cccnc1)...,IC50,NaN,NaN,NaN,0.000000,1158
19,CHEMBL3219552,CHEMBL3217484,CCCOc1ccc(CN2C(=S)NC(=O)C(=C2c3ccccc3)Cc4c(OC)...,IC50,=,61900.0,nM,4.208309,1115
